In [381]:
from collections import Counter
import codecs
import re
import math
from scipy import stats


def mean(values: list, second: list = None) -> float:
    """
    Вычисление среднего значения
    """
    if second is None:
        return sum(values)/len(values)
    else:
        return sum([x*y for x, y in zip(values, second)])/len(values)

    
def R2(X: list, Y:list) -> float:
    """
    Вычисление коэффициента детерминации
    """
    if len(X) > len(Y):
        X = X[:len(Y)]
    elif len(X) < len(Y):
        Y = Y[:len(X)]
    x_mean = mean(X)
    y_mean = mean(Y)
    xy_mean = mean(X, Y)   
    sqrt_x = math.sqrt(D_x)
    sqrt_y = math.sqrt(D_y)
    return math.pow((xy_mean - x_mean * y_mean)/(sqrt_x * sqrt_y), 2)


def my_chi2(O: list, E: list) -> float:
    """
    :param O: Наблюдаемое значение
    :param E: Ожидаемое значение
    :return: Хи2 Пирсона
    """
    chi2_val = 0
    for o, e in zip(O, E):
        chi2_val += ((e - o)**2)/e
    return chi2_val


def math_wait(value: list, probability: list) -> float:
    """
    Математическое ожидание дискретной величины
    :param value: список значений
    :param probability: список соответствующих значениям вероятностей
    :return: M[x]
    """
    return sum([x * p for x, p in zip(value, probability)])


def regular_count(sequence: str, not_summary: bool = True, *symbols: str) -> Counter():
    """
    :param sequence : символьная последовательность без пробелов (исходные данные)
    :param symbols: символы, на основе которых будет осуществляться поиск под-последовательностей
    :param not_summary: True - считать количество элементов для каждой подпоследовательности
    False считать только колличество элементов в подпоследовательности
    :return: соотношение подпоследовательность(ключ) количество совпадений(значение) в виде словаря Counter
    """
    if not_summary:
        return {i: j for i, j in sorted(
            Counter(re.findall('|'.join([f'[{s}]*[{s}]' for s in symbols]), sequence)).items(),
            key=lambda x: x[0])}
    else:
        return {i: j for i, j in sorted(Counter([len(seq) for seq in re.findall(
            '|'.join([f'[{s}]*[{s}]' for s in symbols]), sequence)]).items(), key=lambda x: x[0]) }


def predict(length: int, n: int) -> float:
    """
    Предсказание количества идущих подряд символов для последовательности с известной длинной
    :param length: длинна последовательности
    :param n: количество символов, следующих подрад, для котогоко делаем предсказание
    :return: средняя вероятности встретить под последовательности длинной n в последовательности length
    """
    if length < 0 or n < 0:
        raise ValueError
    return length/2**(n+1)


def limit_series(length: int, predicate, index: int = 0) -> {int, float}:
    """
    Ряд Nn ограниченный условием
    :param predicate: функция - условие ограничения ряда
    :param length: количество элементов в рассматриваемой последовательности
    :param index: минимальная длина пол-последовательности символов, для которой будет рассчитана вероятности
    :return: словарь, где ключи - количество элементов, значения - прогнозируемое количество, отвечающее
    условию predicate
    """
    if length < 0 or index < 0:
        raise ValueError
    temporal = {}
    while True:
        step = predict(length, index)
        if predicate(step):
            temporal[index] = step
            index += 1
        else:
            return temporal

def p_value(chi2: float, free: int) -> tuple:
    """
    Распределение значений хи-квадрата от для df от 1 до 30
    :param chi2: значение хи кваднрата, для которого определяется p-значение
    :param free: Степень свободы
    :return: возвращает пару значений, ограничивающих p-значение в виде кортежа
    """
    __critical_dict = {
        0.995: (0.0000393,  0.0100, 0.0717, 0.207, 0.412, 0.676, 0.989, 1.344, 1.735, 2.156, 2.603, 3.074, 3.565, 4.075,
                4.601, 5.142, 5.697, 6.265, 6.844, 7.434, 8.034, 8.643, 9.260, 9.886, 10.520, 11.160, 11.808, 12.461,
                13.121, 13.787),
        0.975: (0.000982, 0.0506, 0.216, 0.484, 0.831, 1.237, 1.690, 2.180, 2.700, 3.247, 3.816, 4.404, 5.009, 5.629,
                6.262, 6.908, 7.564, 8.231, 8.907, 9.591, 10.283, 10.982, 11.689, 12.401, 13.120, 13.844, 14.573,
                15.308, 16.047, 16.791),
        0.5: (0.455, 1.39, 2.37, 3.36, 4.35, 5.35, 6.35, 7.34, 8.34,  9.34, 10.3, 11.3,  12.3, 13.3, 14.3, 15.3, 16.3,
              17.3, 18.3, 19.3, 20.3, 21.3, 22.3, 23.3, 24.3, 25.3, 26.3, 27.3, 28.3, 29.3),
        0.2: (1.642, 3.219, 4.642, 5.989, 7.289, 8.558, 9.803, 11.030, 12.242, 13.442, 14.631, 15.812, 16.985, 18.151,
              19.311, 20.465, 21.615, 22.760, 23.900, 25.038, 26.171, 27.301, 28.429, 29.553, 30.675, 31.795, 32.912,
              34.027, 35.139, 36.250),
        0.1: (2.706, 4.605, 6.251, 7.779, 9.236, 10.645, 12.017, 13.362, 14.684, 15.987, 17.275, 18.549, 19.812, 21.064,
              22.307, 23.542, 24.769, 25.989, 27.204, 28.412, 29.615, 30.813, 32.007, 33.196, 34.382, 35.563, 36.741,
              37.916, 39.087, 40.256),
        0.05: (3.841, 5.991, 7.815, 9.488, 11.07, 12.592, 14.067, 15.507, 16.919, 18.307, 19.675, 21.026, 22.362,
               23.685, 24.996, 26.296, 27.587, 28.869, 30.144, 31.41),
        0.025: (5.024, 7.378, 9.348, 11.143, 12.833, 14.449, 16.013, 17.535, 19.023, 20.483, 21.920, 23.337, 24.736,
                26.119, 27.488, 28.845, 30.191, 31.526, 32.852, 34.170, 35.479, 36.781, 38.076, 39.364, 40.646, 41.923,
                43.195, 44.461, 45.722, 46.979),
        0.02: (5.412, 7.824, 9.837, 11.668, 13.388, 15.033, 16.622, 18.168, 19.679, 21.161, 22.618, 24.054, 25.472,
               26.873, 28.259, 29.633, 30.995, 32.346, 33.687, 35.020, 36.343, 37.659, 38.968, 40.270, 41.566, 42.856,
               44.140, 45.419, 46.693, 47.962),
        0.01: (6.635, 9.21, 11.345, 13.277, 15.086, 16.812, 18.475, 20.09, 21.66, 23.209, 24.725, 26.217, 27.688,
               29.141, 30.578, 32, 33.409, 34.805, 36.191, 37.566),
        0.005: (7.879, 10.597, 12.838, 14.860, 16.750, 18.548, 20.278, 21.955, 23.589, 25.188, 26.757, 28.300, 29.819,
                31.319, 32.801, 34.267, 35.718, 37.156, 38.582, 39.997, 41.401, 42.796, 44.181, 45.559, 46.928, 48.290,
                49.645, 50.993, 52.336, 52.336),
        0.002: (9.550, 12.429, 14.796, 16.924, 18.907, 20.791, 22.601, 24.352, 26.056, 27.722, 29.354, 30.957, 32.535,
                34.091, 35.628, 37.146, 38.648, 40.136, 41.610, 43.072, 44.522, 45.962, 47.391, 48.812, 50.223, 51.627,
                53.023, 54.411, 55.792, 57.167),
        0.001: (10.828, 13.816, 16.266, 18.467, 20.515, 22.458, 24.322, 26.124, 27.877, 29.588, 31.264, 32.909, 34.528,
                36.123, 37.697, 39.252, 40.790, 42.312, 43.820, 45.315, 46.797, 48.268, 49.728, 51.179, 52.620, 54.052,
                55.476, 56.892, 58.301, 59.703)
    }
    if __critical_dict[0.995][free] > chi2:
        return 1, 0.995
    for i, j in zip(list(__critical_dict.keys())[::1], list(__critical_dict.keys())[1:]):
        if __critical_dict[j][free] > chi2:
            return i, j
    return 0.001, 0


def print_dicts(value: dict):
    """
    Вывод на экран словаря, значения которого - тоже словари
    """
    for i, j in value.items():
        print(i)
        for k, v in j.items():
            print(f"{k} {v}")

In [382]:
if __name__ == '__main__':
    # Парсим файл с исходными данными
    simple_parser = {}
    with codecs.open(
                'C:/Users/RTFE4/Desktop/ИТАСУ/ИВТ/2 Семестр/Специальные главы математики часть 2/ДЗ/ДЗ8/0and1.txt',
            'r', 'utf-8') as reader:
        temp = []
        for i in reader:
            temp.append(i)
            if len(temp) == 2:
                value = temp.pop()
                name = temp.pop()
                simple_parser[name.partition('\r')[0]] = re.sub('[\[\],]', '', value.partition('\r')[0])
    # Получить предсказание для каждой последовательности
    predicted_values = {i: limit_series(len(j), lambda x: x >= 1, 1) for i, j in simple_parser.items()}
    # Получить эксперементальные значения для последовательности
    experimental_values = {i: regular_count(j, False, '0', '1') for i, j in simple_parser.items()}

In [383]:
# расчет хи-квадрата и p-значения + коэффициент детерминации (R2) 
if __name__ == '__main__':    
    for o, e in zip(experimental_values.items(), predicted_values.values()):
        ch2 = my_chi2(list(o[1].values()), list(e.values()))
        p = p_value(ch2, min(len(e), len(o[1].values())))
        if p is not None:
            print(f"{o[0]} \t R2~{round(R2(list(o[1].values()),list(e.values())), 2)}" +
                  f" \t ch2 = {ch2:.2f} \t {p[0]} > p > {p[1]}")
        else:
            print(f"{o[0]} \t R2~{round(R2(list(o[1].values()), list(e.values())), 2)}" + 
                  f" \t ch2 = {ch2:.2f} \t p-значение не определено, ch2 слишком большой!")

Ефремова Анастасия 	 R2~0.0 	 ch2 = 19.86 	 0.002 > p > 0.001
Олеся Щербакова 	 R2~0.0 	 ch2 = 60.55 	 0.001 > p > 0
Вячеслав Ермаков 	 R2~0.0 	 ch2 = 20.20 	 0.002 > p > 0.001
Костромин Денис 	 R2~0.0 	 ch2 = 14.52 	 0.02 > p > 0.01
Виктория Коротаева 	 R2~0.0 	 ch2 = 12.78 	 0.05 > p > 0.025
Марина Шипунова 	 R2~0.4 	 ch2 = 473.84 	 0.001 > p > 0
Анастасия Чёрная 	 R2~0.01 	 ch2 = 102.31 	 0.001 > p > 0


In [384]:
print("Прогнозируемые значения:")
print_dicts(predicted_values)
print("Реальные значения:")
print_dicts(experimental_values)

Прогнозируемые значения:
Ефремова Анастасия
1 28.0
2 14.0
3 7.0
4 3.5
5 1.75
Олеся Щербакова
1 25.0
2 12.5
3 6.25
4 3.125
5 1.5625
Вячеслав Ермаков
1 25.0
2 12.5
3 6.25
4 3.125
5 1.5625
Костромин Денис
1 25.0
2 12.5
3 6.25
4 3.125
5 1.5625
Виктория Коротаева
1 24.75
2 12.375
3 6.1875
4 3.09375
5 1.546875
Марина Шипунова
1 250.25
2 125.125
3 62.5625
4 31.28125
5 15.640625
6 7.8203125
7 3.91015625
8 1.955078125
Анастасия Чёрная
1 93.75
2 46.875
3 23.4375
4 11.71875
5 5.859375
6 2.9296875
7 1.46484375
Реальные значения:
Ефремова Анастасия
1 50
2 17
3 8
4 1
Олеся Щербакова
1 62
2 4
3 6
4 3
Вячеслав Ермаков
1 45
2 10
3 10
5 1
Костромин Денис
1 35
2 23
3 5
4 1
Виктория Коротаева
1 42
2 12
3 5
4 2
5 2
Марина Шипунова
1 574
2 144
3 27
4 8
5 4
6 1
Анастасия Чёрная
1 185
2 41
3 28
4 2
5 2
6 1
